## 빅데이터 분석기사 모의고사 1회차
- https://www.datamanim.com/dataset/practice/q1.html

---
### 작업 1유형
- 데이터 출처 : https://archive.ics.uci.edu/ml/datasets/Bank+Marketing (후처리 작업)
- 데이터 설명 : 은행의 전화 마케팅에 대해 고객의 반응 여부
- dataurl : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv
---

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv")
print(df.shape)
df.head()

(12870, 17)


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


#### 1. 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 인원을 가진 나이대는? (0-9 : 0 , 10-19 : 10)

In [13]:
df['age_new'] = df['age'] // 10 * 10
display(df['age_new'].value_counts())

result = df['age_new'].value_counts().index[0]
print(result)

,count
age_new,
30,5056
40,3198
50,2244
20,1638
60,460
70,193
80,57
10,18
90,6


30


#### 2. 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 나이대 구간의 인원은 몇명인가?

In [12]:
result = df['age_new'].value_counts().values[0]
print(result)

5056


#### 3. 나이가 25살 이상 29살 미만인 응답 고객들중 housing컬럼의 값이 yes인 고객의 수는?

In [17]:
cond1 = df['age'] >= 25
cond2 = df['age'] < 29
cond3 = df['housing'] == 'yes'

result = df[cond1 & cond2 & cond3].shape[0]
print(result)

504


#### 4. numeric한 값을 가지지 않은 컬럼들중 unique한 값을 가장 많이 가지는 컬럼은?

In [34]:
obj_cols = df.select_dtypes(exclude='number').columns
unique_cnt = df[obj_cols].nunique()
display(unique_cnt)

print("job, month")

,0
job,12
marital,3
education,4
default,2
housing,2
loan,2
contact,3
month,12
poutcome,4
y,2


job, month


#### 5. balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID값을 기준으로 내림차순 정렬했을때 상위 100개 데이터의 balance값의 평균은?

In [38]:
mean_bal = df['balance'].mean()
df_new = df[df['balance'] >= mean_bal]
result = df_new.sort_values(by='ID', ascending = False)[:100]['balance'].mean()
print(result)

3473.73


#### 6. 가장 많은 광고를 집행했던 날짜는 언제인가? (데이터 그대로 일(숫자),달(영문)으로 표기)

In [43]:
result = df[['day','month']].value_counts()
display(result)
print(result.index[0])

day  month
15   may      301
14   may      283
13   may      257
7    may      239
21   nov      221
             ... 
     sep        1
26   sep        1
5    sep        1
4    oct        1
27   dec        1
Name: count, Length: 303, dtype: int64

(np.int64(15), 'may')


#### ✅ 7. 데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성을 검정하고자 한다. 샤피로 검정의 p-value값을 구하여라



In [51]:
df_unknown = df[df['job'] == 'unknown']

from scipy.stats import shapiro
print(shapiro(df_unknown['age']))
print(shapiro(df_unknown['age'])[1])

ShapiroResult(statistic=np.float64(0.978472096015593), pvalue=np.float64(0.1961223594771469))
0.1961223594771469


#### 8. age와 balance의 상관계수를 구하여라

In [58]:
result = df[['age', 'balance']].corr().iloc[0,1]
print(result)

0.10198734763981462


#### ✅ 9. y 변수와 education 변수는 독립인지 카이제곱검정을 통해 확인하려한다. p-value값을 출력하라

In [64]:
v = pd.crosstab(df.y,df.education)
display(v)

from scipy.stats import chi2_contingency
chi2 , p ,dof, expected = chi2_contingency(v)
print(p)

education,primary,secondary,tertiary,unknown
y,,,,
no,1424,4555,2559,365
yes,456,1813,1516,182


7.901201277473551e-29


#### ✅10. 각 job에 따라 divorced/married 인원의 비율을 확인 했을 때 그 값이 가장 높은 값은?

In [74]:
## 그룹바이 데이터셋 생성
t = df.groupby(['job','marital']).size().reset_index()

## 피봇팅
pivotdf = t.pivot_table(index='job',columns='marital')[0]
pivotdf = pivotdf.fillna(0)

## 이혼 비율 구하기
pivotdf['ratio'] = pivotdf['divorced'] / pivotdf['married']
display(pivotdf.sort_values('ratio',ascending=False))

result = pivotdf['ratio'].max()
print(result)

marital,divorced,married,single,ratio
job,,,,
unemployed,62.0,219.0,133.0,0.283105
retired,157.0,572.0,41.0,0.274476
admin.,207.0,762.0,495.0,0.271654
services,138.0,584.0,321.0,0.236301
technician,246.0,1115.0,780.0,0.220628
management,323.0,1580.0,955.0,0.204430
housemaid,49.0,245.0,40.0,0.200000
entrepreneur,44.0,272.0,67.0,0.161765
self-employed,39.0,284.0,131.0,0.137324


0.2831050228310502


---
### 작업 2유형 : 전화마케팅에 대한 고객 반응 예측 모형 구축 (평가: ROC score)

- 데이터 출처 : https://archive.ics.uci.edu/ml/datasets/Bank+Marketing (후처리 작업)
- 데이터 설명 : 은행의 전화 마케팅에 대해 고객의 반응 여부
- train : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv
- test : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv
- submission : https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv
---

In [75]:
train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv")
submission = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv")

print(train.shape, test.shape, submission.shape)

display(train.head(2))
display(test.head(2))
display(submission.head(2))

(12870, 17) (4291, 16) (4291, 2)


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
0,53608,32,management,single,tertiary,no,12569,no,no,cellular,1,jul,2,295,2,success
1,51055,25,services,single,secondary,no,801,no,no,cellular,5,jun,2,-1,0,unknown


,ID,predict
0,53608,0.0
1,51055,0.0


In [84]:
# 결측치 확인 : 없음
# y, X 지정
y = train['y']
X = train.drop(['ID','y'],axis=1)
# categorical columns --> dummies
Xd = pd.get_dummies(X)

# 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(Xd, y, test_size=0.2, stratify=y, random_state=42)

# 모델링
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
pred = model.predict_proba(X_valid)

# 평가
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_valid, pred[:,1]))

0.7823038192834126


In [89]:
# test 데이터 예측
test_pred = model.predict_proba(pd.get_dummies(test.drop(columns=['ID'])))
submission['predict'] = test_pred[:, 1]
display(submission)

submission.to_csv('submission_yb.csv', index=False)

,ID,predict
0,53608,0.80
1,51055,0.95
2,52573,0.05
3,50458,0.12
4,52272,0.27
...,...,...
4286,50332,0.14
4287,51293,0.80
4288,54023,0.14
4289,53775,0.67
